# Convert BCL to FASTQ with UMI preprocessing
- convert HiSeq2500 raw data to fastq and perform preprocesing for unique molecular identifiers (UMI)


1. (010) parse samplesheet.csv to generate barcodefile.txt and library_params_laneN.txt
1. (020-030) run Picard ExtracrtIlluminaBarcodes
1. (040-050) run Picard IlluminaBasecallsToSam (generate unaligned bam [u.bam] files)
1. (060-070) run Picard SamToFastq

In [7]:
cd /peta/top/raw_data/HiSeq2500/190909_D00385_0273_BHY2VKBCX2_UMI
ls -al

合計 924
drwxrwxr-x 14 misshie misshie    8192  9月 17 14:13 .
drwxrwxr-x 27 misshie genetics   8192  3月 11 16:00 ..
-rwxrwxr-x  1 misshie misshie    2872  4月 23  2019 010_parse_samplesheet-Barcode1.rb
-rwxrwxr-x  1 misshie misshie     637  4月 22  2019 020_deploy-ExtractIlluminaBarcodes.rb
-rwxrwxr-x  1 misshie misshie     394  4月 23  2019 030_submit-ExtractIlluminaBarcodes.rb
-rwxrwxr-x  1 misshie misshie     795  4月 18  2019 040_deploy_IlmnBasecallToSam.rb
-rwxrwxr-x  1 misshie misshie     378  4月 23  2019 050_submit-BasecallToSam.rb
-rwxrwxr-x  1 misshie misshie    1015  4月 25  2019 060_deploy_SamToFastq.rb
-rwxrwxr-x  1 misshie misshie     390  9月 17 12:29 070_submit-SamToFastq.rb
drwxrwxr-x  2 misshie misshie       6  9月 10  2019 BarcodeImages
drwxr-xr-x  2 misshie misshie    4096  9月 11  2019 Barcodes_outputdir_lane1
drwxr-xr-x  2 misshie misshie    4096  9月 11  2019 Barcodes_outputdir_lane2
-rw-rw-r--  1 misshie misshie      45  9月 10  2019 Basecalling_Netcopy_complete.txt
-rw-rw-r

In [3]:
cat SampleSheet.csv

FCID,Lane,SampleID,SampleRef,Index,Index2,Description,Control,Recipe,Operator,SampleProject
HY2VKBCX2,1,NGSK_dummy_dummy_dummy,hg19,GCCAAGAC,NNNNNNNNNN,NGSK_dummy_dummy_dummy,N,Agilent_indexing,dummy,Project1
HY2VKBCX2,1,NGSK_dummy_dummy_dummyFather,hg19,CTGTAGCC,NNNNNNNNNN,NGSK_dummy_dummy_dummyFather,N,Agilent_indexing,dummy,Project1
HY2VKBCX2,1,NGSK_dummy_dummy_dummyMother,hg19,CGCTGATC,NNNNNNNNNN,NGSK_dummy_dummy_dummyMother,N,Agilent_indexing,dummy,Project1
HY2VKBCX2,2,NGSK_dummy_dummy_dummy,hg19,GCCAAGAC,NNNNNNNNNN,NGSK_dummy_dummy_dummy,N,Agilent_indexing,dummy,Project1
HY2VKBCX2,2,NGSK_dummy_dummy_dummyFather,hg19,CTGTAGCC,NNNNNNNNNN,NGSK_dummy_dummy_dummyFather,N,Agilent_indexing,dummy,Project1
HY2VKBCX2,2,NGSK_dummy_dummy_dummyMother,hg19,CGCTGATC,NNNNNNNNNN,NGSK_dummy_dummy_dummyMother,N,Agilent_indexing,dummy,Project1


In [2]:
cat 010_parse_samplesheet-Barcode1.rb

#!/usr/bin/env ruby
require 'optparse'
require 'fileutils'

class Klass
  SSHEET = "SampleSheet.csv"
  BCFILE = "barcodefile.txt"
  LIBPARFILE = "library_params"
  UNALIGNED = "Unaligned"

  BCFILE_HEADER = "barcode_name\tlibrary_name\tbarcode_sequence_1"
  BcRec = Struct.new(:bcname, :libname, :bcseq1, :bcseq2)
  LIBPAR_HEADER = "BARCODE_1\tSAMPLE_ALIAS\tLIBRARY_NAME\tOUTPUT"
  ParRec = Struct.new(:bcseq1, :sample, :libname, :outbam)
  SSheetRec = Struct.new(:fcid, :lane, :sampleid, :sampleref,
                         :index1, :index2,
                         :description, :control, :recipe, :operator, :project)

  attr_reader :opts, :ssheets
  
  def initialize(opts)
    @opts = opts
  end

  def load_samplesheet
    @ssheets = Array.new
    File.readlines(SSHEET).each_with_index do |row, nrow|
      row.chomp!
      next if nrow.zero?
      next if row.gsub(/,/, "").empty?
      @ssheets << SSheetRec.new(*row.split(","))
    end
  end

  def make_bcfile
    open(BCFILE, "w") do |f

In [4]:
cat barcodefile.txt

barcode_name	library_name	barcode_sequence_1
NGSK_dummy_dummy_dummy	NGSK_dummy_dummy_dummy	GCCAAGACNNNNNNNNNN	
NGSK_dummy_dummy_dummyFather	NGSK_dummy_dummy_dummyFather	CTGTAGCCNNNNNNNNNN	
NGSK_dummy_dummy_dummyMother	NGSK_dummy_dummy_dummyMother	CGCTGATCNNNNNNNNNN	


In [8]:
cat library_params_lane1.txt

BARCODE_1	SAMPLE_ALIAS	LIBRARY_NAME	OUTPUT
GCCAAGACNNNNNNNNNN	NGSK_dummy_dummy_dummy	NGSK_dummy_dummy_dummy	Unaligned/Project_dummy-P/Sample_NGSK_dummy_dummy_dummy/NGSK_dummy_dummy_dummy_GCCAAGACNNNNNNNNNN_L001.u.bam
CTGTAGCCNNNNNNNNNN	NGSK_dummy_dummy_dummyFather	NGSK_dummy_dummy_dummyFather	Unaligned/Project_dummy-P/Sample_NGSK_dummy_dummy_dummyFather/NGSK_dummy_dummy_dummyFather_CTGTAGCCNNNNNNNNNN_L001.u.bam
CGCTGATCNNNNNNNNNN	NGSK_dummy_dummy_dummyMother	NGSK_dummy_dummy_dummyMother	Unaligned/Project_dummy-P/Sample_NGSK_dummy_dummy_dummyMother/NGSK_dummy_dummy_dummyMother_CGCTGATCNNNNNNNNNN_L001.u.bam
N	Unmached	Unmached	Unaligned/Undetermined_indices/Sample_lane1/lane1_Undetermined_L001.u.bam


In [9]:
cat Templates/sge-ExtractIlluminaBarcodes.sh.erb

#!/bin/bash
#$ -S /bin/bash
#$ -N <%= jobname %>
#$ -cwd
#$ -m abe
#$ -M root
#$ -j yes
#$ -pe smp 32
#$ -q all.q
set -euo pipefail

: "embeded parameters", && {
    lane=<%= lane %>
}

: "file paths and settings" && {
    rs="100T8B10M100T"
    threads=8
    jar=/xcatopt/picard-tools-2.19.0/picard.jar
    basecalldir=./Data/Intensities/BaseCalls
    java=/xcatopt/jre1.8.0_212/bin/java
}

: "run picard" && {

    mkdir -p Barcodes_outputdir_lane${lane}
    ${java} -Xmx8g \
	 -jar ${jar} \
	 ExtractIlluminaBarcodes \
	 BASECALLS_DIR=${basecalldir} \
	 BARCODE_FILE=barcodefile.txt \
	 READ_STRUCTURE=${rs} \
	 LANE=${lane} \
	 OUTPUT_DIR=Barcodes_outputdir_lane${lane} \
	 COMPRESS_OUTPUTS=true \
	 METRICS_FILE=barcode_metrics_lane${lane}.txt \
	 NUM_PROCESSORS=${threads} \
	 > barcode_lane${lane}.log 2>&1
}


In [10]:
cat 020_deploy-ExtractIlluminaBarcodes.rb

#!/usr/bin/env ruby
# Ruby 2.5 and later

require 'erb'
require 'fileutils'
require 'time'

class DeploySamples
  NUM_LANES = 2
  ERBFILE   = "./Templates/sge-ExtractIlluminaBarcodes.sh.erb"
  SUBMITTER = "sge-ExtractIlluminaBarcodes_lane<%>.sh"
  
  def run
    STDOUT.sync = true
    (1..NUM_LANES).each do |lane|      
      submitter = SUBMITTER.sub(/<%>/, lane.to_s)
      template = ERB.new File.read(ERBFILE)
      open(submitter, 'w') do |fout|
        fout.puts template.result_with_hash \
          jobname: "extbc#{lane}", \
          lane: lane
      end
    end
  end
    
end

if $0 == __FILE__
  DeploySamples.new.run
end


In [11]:
cat sge-ExtractIlluminaBarcodes_lane1.sh

#!/bin/bash
#$ -S /bin/bash
#$ -N extbc1
#$ -cwd
#$ -m abe
#$ -M root
#$ -j yes
#$ -pe smp 32
#$ -q all.q
set -euo pipefail

: "embeded parameters", && {
    lane=1
}

: "file paths and settings" && {
    rs="100T8B10M100T"
    threads=8
    jar=/xcatopt/picard-tools-2.19.0/picard.jar
    basecalldir=./Data/Intensities/BaseCalls
    java=/xcatopt/jre1.8.0_212/bin/java
}

: "run picard" && {

    mkdir -p Barcodes_outputdir_lane${lane}
    ${java} -Xmx8g \
	 -jar ${jar} \
	 ExtractIlluminaBarcodes \
	 BASECALLS_DIR=${basecalldir} \
	 BARCODE_FILE=barcodefile.txt \
	 READ_STRUCTURE=${rs} \
	 LANE=${lane} \
	 OUTPUT_DIR=Barcodes_outputdir_lane${lane} \
	 COMPRESS_OUTPUTS=true \
	 METRICS_FILE=barcode_metrics_lane${lane}.txt \
	 NUM_PROCESSORS=${threads} \
	 > barcode_lane${lane}.log 2>&1
}


In [12]:
cat sge-BasecallToSam_lane1.sh

#!/bin/bash
#$ -S /bin/bash
#$ -N bc2sam1
#$ -cwd
#$ -m abe
#$ -M root
#$ -j yes
#$ -pe smp 32
#$ -q all.q
set -euo pipefail

: "embeded parameters", && {
    barcodedir=Barcodes_outputdir_lane1
    lane=1
    libparams=library_params_lane1.txt
}

: "file paths and settings" && {
    rs="100T8B10M100T"
    threads=8
    jar=/xcatopt/picard-tools-2.19.0/picard.jar
    basecalldir=./Data/Intensities/BaseCalls
    scratch=/peta/btm/scratch
    seqcenter="GenkenIden"
    machineid="HWI-D00385"
    java=/xcatopt/jre1.8.0_212/bin/java
}

: "run picard" && {
    ${java} -Xmx64g \
	 -jar ${jar} \
	 IlluminaBasecallsToSam \
	 BASECALLS_DIR=${basecalldir} \
	 BARCODES_DIR=${barcodedir} \
	 LANE=${lane} \
	 READ_STRUCTURE=${rs} \
	 SEQUENCING_CENTER=${seqcenter} \
	 RUN_BARCODE=${machineid} \
	 LIBRARY_PARAMS=${libparams} \
	 ADAPTERS_TO_CHECK=INDEXED \
	 ADAPTERS_TO_CHECK=DUAL_INDEXED \
	 ADAPTERS_TO_CHECK=PAIRED_END \
	 MOLECULAR_INDEX_TAG=RX \
	 MOLECULAR_INDEX_BASE_QUALITY_TAG=QX \
	 TMP_DIR=

In [13]:
cat 060_deploy_SamToFastq.rb

#!/usr/bin/env ruby
# Ruby 2.5 and later

require 'erb'
require 'fileutils'
require 'time'

class DeploySamples

  ERBFILE   = "./Templates/sge-SamToFastq.sh.erb"
  SUBMITTER = "sge-SamToFastq_lane<%>.sh"
  
  def run
    STDOUT.sync = true
    Dir["Unaligned/Project_*/Sample_*/*.u.bam"].sort.each_with_index do |ubam, idx|
      warn "Processing #{File.basename(ubam)}.."
      sects = File.basename(ubam).split("_")
      lane = sects[-1].sub(/^L00/,"").sub(/\.u\.bam$/, "")
      fastq1 = ubam.sub(/\.u\.bam$/,"_R1_001.fastq.gz")
      fastq2 = ubam.sub(/\.u\.bam$/,"_R2_001.fastq.gz")
      submitter = SUBMITTER.sub(/<%>/, "#{lane}-#{idx}")
      template = ERB.new File.read(ERBFILE)
      open(submitter, 'w') do |fout|
        fout.puts template.result_with_hash \
          jobname: "b2fq#{lane}-#{idx}", \
          inputbam: ubam, \
          fastq1: fastq1, \
          fastq2: fastq2, \
          log: "#{submitter}.log"
      end
    end
  end
    
end

if $0 == __FILE__
  DeploySampl

In [14]:
cat Templates/sge-SamToFastq.sh.erb

#!/bin/bash
#$ -S /bin/bash
#$ -N <%= jobname %>
#$ -cwd
#$ -m abe
#$ -M root
#$ -j yes
#$ -pe smp 8
#$ -q all.q
set -euo pipefail

: "embeded parameters", && {
    inputbam=<%= inputbam %>
    fastq1=<%= fastq1 %>
    fastq2=<%= fastq2 %>
    log=<%= log %>
}

: "file paths and settings" && {
    jar=/xcatopt/picard-tools-2.19.0/picard.jar
    scratch=/peta/btm/scratch
    #java=/xcatopt/jre1.8.0_212/bin/java
    java=java
    javaopt="-Xmx16g"
}

: "run picard" && {
    ${java} ${javaopt} \
	 -jar ${jar} \
	 SamToFastq \
	 INPUT=${inputbam} \
	 FASTQ=${fastq1} \
	 SECOND_END_FASTQ=${fastq2} \
	 INCLUDE_NON_PF_READS=false \
	 CLIPPING_ATTRIBUTE=XT \
	 CLIPPING_ACTION=X \
	 CLIPPING_MIN_LENGTH=0 \
	 TMP_DIR=${scratch} \
	 > ${log} 2>&1
}


In [15]:
cat sge-SamToFastq_lane1-0.sh

#!/bin/bash
#$ -S /bin/bash
#$ -N b2fq1-0
#$ -cwd
#$ -m abe
#$ -M root
#$ -j yes
#$ -pe smp 8
#$ -q all.q
set -euo pipefail

: "embeded parameters", && {
    inputbam=Unaligned/Project_dummy/Sample_dummy_Fa/dummy_Fa_ACACGACCNNNNNNNNNN_L001.u.bam
    fastq1=Unaligned/Project_dummy/Sample_dummy_Fa/dummy_Fa_ACACGACCNNNNNNNNNN_L001_R1_001.fastq.gz
    fastq2=Unaligned/Project_dummy/Sample_dummy_Fa/dummy_Fa_ACACGACCNNNNNNNNNN_L001_R2_001.fastq.gz
    log=sge-SamToFastq_lane1-0.sh.log
}

: "file paths and settings" && {
    jar=/xcatopt/picard-tools-2.19.0/picard.jar
    scratch=/peta/btm/scratch
    #java=/xcatopt/jre1.8.0_212/bin/java
    java=java
    javaopt="-Xmx16g"
}

: "run picard" && {
    ${java} ${javaopt} \
	 -jar ${jar} \
	 SamToFastq \
	 INPUT=${inputbam} \
	 FASTQ=${fastq1} \
	 SECOND_END_FASTQ=${fastq2} \
	 INCLUDE_NON_PF_READS=false \
	 CLIPPING_ATTRIBUTE=XT \
	 CLIPPING_ACTION=X \
	 CLIPPING_MIN_LENGTH=0 \
	 TMP_DIR=${scratch} \
	 > ${log} 2>&1
}


In [16]:
ls -al Unaligned/Project_dummy/Sample_dummy_Fa

合計 11669216
drwxrwxr-x 2 misshie misshie       4096  9月 17 09:32 .
drwxrwxr-x 5 misshie misshie        100  9月 11  2019 ..
-rw-r--r-- 1 misshie misshie 3252513941  9月 17 14:03 dummy_Fa_ACACGACCNNNNNNNNNN_L001.u.bam
-rw-r--r-- 1 misshie misshie 1367129265  9月 17 14:26 dummy_Fa_ACACGACCNNNNNNNNNN_L001_R1_001.fastq.gz
-rw-r--r-- 1 misshie misshie 1427475323  9月 17 14:26 dummy_Fa_ACACGACCNNNNNNNNNN_L001_R2_001.fastq.gz
-rw-r--r-- 1 misshie misshie 3169524297  9月 17 13:59 dummy_Fa_ACACGACCNNNNNNNNNN_L002.u.bam
-rw-r--r-- 1 misshie misshie 1337381279  9月 17 14:27 dummy_Fa_ACACGACCNNNNNNNNNN_L002_R1_001.fastq.gz
-rw-r--r-- 1 misshie misshie 1395239388  9月 17 14:27 dummy_Fa_ACACGACCNNNNNNNNNN_L002_R2_001.fastq.gz
